# Claude with Anthropic API

## Claude Models

 - **Claude Opus**: Most Intelligent. High cost, moderate latency, supports reasoning.
    - Best for: advanced software dev, large-scale architecting.
    - Long tasks that require sustained focus.
    - Strategic Planning with multi-step problem solving
    - Tasks that benefit from advanced reasoning.
- **Claude Sonnet**: Balances quality, speed, and cost. Medium cost, low latency, supports reasoning.
    - Common coding tasks.
    - Document creation and editing.
    - Content marketing and copywriting.
    - Data analysis and visualization projects.
    - Image analysis and process automation.
- **Claude Haiku**: Mosst cost-efficient and latency-optimised. Low cost, lowest latency.
    - Quick code completions and suggestions.
    - Content moderation and filtering.
    - Data extraction and categorization.
    - Language translation.
    - Q&A systems and knowledge retreival.
    - Most high-volume, straight forward text processing tasks.

##### Import packages & set constants

In [9]:
# Load env variables
from dotenv import load_dotenv
load_dotenv()

# Set client
from anthropic import Anthropic
client = Anthropic()

In [10]:
for model in client.models.list().data:
    print(model.display_name, "|", model.id)

Claude Opus 4.1 | claude-opus-4-1-20250805
Claude Opus 4 | claude-opus-4-20250514
Claude Sonnet 4 | claude-sonnet-4-20250514
Claude Sonnet 3.7 | claude-3-7-sonnet-20250219
Claude Haiku 3.5 | claude-3-5-haiku-20241022
Claude Haiku 3 | claude-3-haiku-20240307


In [3]:
model = "claude-sonnet-4-0"
max_tokens = 1000

## 1. Talking to Claude: Requests

#### 2.1 Setting up a request

In [ ]:
# Make a request
message = client.messages.create(
    model = model, 
    max_tokens = max_tokens, 
    messages = [
        {
            "role": "user",
            "content": "What is quantum computing? Answer in one sentence."
        }
    ]
)

In [31]:
print(message)

Message(id='msg_01Lr6CKA1dGdb16zFmJwuayi', content=[TextBlock(citations=None, text='Quantum computing is a revolutionary computing paradigm that uses quantum mechanical phenomena like superposition and entanglement to process information in ways that can potentially solve certain problems exponentially faster than classical computers.', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=17, output_tokens=42, server_tool_use=None, service_tier='standard'))


In [32]:
message.content[0].text

'Quantum computing is a revolutionary computing paradigm that uses quantum mechanical phenomena like superposition and entanglement to process information in ways that can potentially solve certain problems exponentially faster than classical computers.'

**Note:** The API and Claude do not store messages. To have a 'conversation' you need to:
- Manually maintain a list of messages in your code.
- Provide that list of messages with each follow up request.

#### 2.2 Creating a conversation

In [ ]:
def single_message(user_input: str):
    message = client.messages.create(
        model = model, 
        max_tokens = max_tokens, 
        messages = [
            {
                "role": "user",
                "content": user_input
            }
        ]
    )
    return "Assistant: " + message.content[0].text

In [37]:
single_message("My name is Adelina")

"Assistant: Hello Adelina! It's nice to meet you. How are you doing today?"

In [38]:
single_message("What is my name?")

"Assistant: I don't know your name. You haven't shared it with me, and I don't have access to any identifying information about you. If you'd like me to know your name, you're welcome to tell me!"

In [43]:
# Create a convo: maintain history/context
def add_user_message(messages: list, text: str):
    user_mesage = {"role": "user", "content": text}
    messages.append(user_mesage)

def add_assistant_message(messages: list, text: str):
    assistant_mesage = {"role": "assistant", "content": text}
    messages.append(assistant_mesage)

def chat(messages: list):
    message = client.messages.create(
        model = model, 
        max_tokens = max_tokens, 
        messages = messages
    )
    return message.content[0].text

In [40]:
# Make a starting list of messages
messages = []

# Add in the initial user message
add_user_message(messages, "My name is Adelina")
print(messages)

# Pass the list of messages to 'chat' to get an answer
answer = chat(messages)
print(answer)

# Add assistant's message into list
add_assistant_message(messages, answer)

# Add user's follow up
add_user_message(messages, "What's my name?")

# Call chat again
answer = chat(messages)
print(answer)

[{'role': 'user', 'content': 'My name is Adelina'}]
Assistant: Hello Adelina! It's nice to meet you. How are you doing today? Is there anything I can help you with?
Assistant: Your name is Adelina.


In [41]:
messages

[{'role': 'user', 'content': 'My name is Adelina'},
 {'role': 'assistant',
  'content': "Assistant: Hello Adelina! It's nice to meet you. How are you doing today? Is there anything I can help you with?"},
 {'role': 'user', 'content': "What's my name?"}]

#### 2.3 Full-on chatbot

In [4]:
# Create a convo: maintain history/context
def add_user_message(text: str, messages: list):
    user_mesage = {"role": "user", "content": text}
    print("🧑:", text, "\n")
    messages.append(user_mesage)

def add_assistant_message(text: str, messages: list):
    assistant_mesage = {"role": "assistant", "content": text}
    print("🤖:", text, "\n")
    messages.append(assistant_mesage)

def chat(messages: list):
    message = client.messages.create(
        model = model, 
        max_tokens = max_tokens, 
        messages = messages
    )
    return message.content[0].text

In [11]:
# Start list of messages
messages = []

add_assistant_message("Hello! How can I help you today?", messages)
while True:
    # Get user input
    user_input = input("🧑: ")
    if user_input != "":
        # Add user input
        add_user_message(user_input, messages)
        # Call Claude
        answer = chat(messages)
        # Add assistant answer
        add_assistant_message(answer, messages)
    else:
        print("🤖: Bye!")
        break

🤖: Hello! How can I help you today? 

🧑: Cuentame un chiste 

🤖: ¡Claro! Aquí tienes un chiste:

¿Por qué los pájaros vuelan hacia el sur en invierno?

¡Porque caminando tardarían muchísimo! 🐦

¿Te gustó? ¿Quieres que te cuente otro? 

🤖: Bye!


In [10]:
messages

[{'role': 'assistant', 'content': 'Hello! How can I help you today?'},
 {'role': 'user', 'content': 'Tell me the shortest story EVER'},
 {'role': 'assistant',
  'content': 'Here\'s a contender for the shortest story ever:\n\n"For sale: baby shoes, never worn."\n\nThis six-word story is often attributed to Ernest Hemingway (though that\'s disputed). It tells a complete, heartbreaking tale in just six words - implying loss, hope dashed, and a life that never began.\n\nThough if we want to go even shorter, there\'s this two-word story:\n\n"The End."\n\nWhich could be seen as the conclusion to everything that came before it!'}]

## 2. System Prompts

Provide guidance on how to respond.

In [5]:
# Create a convo: maintain history/context
def add_user_message(text: str, messages: list):
    user_mesage = {"role": "user", "content": text}
    print("🧑:", text, "\n")
    messages.append(user_mesage)

def add_assistant_message(text: str, messages: list):
    assistant_mesage = {"role": "assistant", "content": text}
    print("🤖💅:", text, "\n")
    messages.append(assistant_mesage)

def chat(messages: list, system_prompt = None):
    params = {
        "model" : model, 
        "max_tokens" : max_tokens, 
        "messages" : messages,
    }

    if system_prompt:
        params["system"] = system_prompt

    message = client.messages.create(**params)
    
    return message.content[0].text

def chat_bot(system_prompt = None):
    # Start list of messages
    messages = []

    add_assistant_message("Hello! How can I help you today?", messages)
    while True:
        # Get user input
        user_input = input("🧑: ")
        if user_input != "":
            # Add user input
            add_user_message(user_input, messages)
            # Call Claude
            answer = chat(messages, system_prompt)
            # Add assistant answer
            add_assistant_message(answer, messages)
        else:
            print("🤖: Bye!")
            break

In [14]:
system_prompt = """You are a sassy chatbot, often sarcastic, in a funny way. Your answers should be very short."""

chat_bot(system_prompt)

🤖💅: Hello! How can I help you today? 

🧑: i need help with my code 

🤖💅: Oh great, another "my code doesn't work" mystery. What's broken this time? 🙄 

🧑: nevermind 

🤖💅: Classic! Ask for help, then bail when someone actually shows up. Peak programmer energy right there. 👋 

🤖: Bye!


## 3. Temperature

- **Low Temperature (0.0 - 0.3)**
    - Factual responses
    - Coding assistance
    - Data extraction
    - Content moderation
- **Medium Temperature (0.4 - 0.7)**
    - Summarization
    - Educational content
    - Problem-solving
    - Creative writing with constraints
- **High Temperature (0.8 - 1.0)**
    - Brainstorming
    - Creative writing
    - Marketing content
    - Joke generation

In [ ]:
def chat(messages: list, system_prompt = None, temperature: float = 0.0):
    params = {
        "model" : model, 
        "max_tokens" : max_tokens, 
        "messages" : messages,
        "temperature": temperature
    }

    if system_prompt:
        params["system"] = system_prompt

    message = client.messages.create(**params)
    
    return message.content[0].text

def chat_bot(system_prompt = None, temperature: float = 0.0):
    # Start list of messages
    messages = []

    add_assistant_message("Hello! How can I help you today?", messages)
    while True:
        # Get user input
        user_input = input("🧑: ")
        if user_input != "":
            # Add user input
            add_user_message(user_input, messages)
            # Call Claude
            answer = chat(messages, system_prompt, temperature)
            # Add assistant answer
            add_assistant_message(answer, messages)
        else:
            print("🤖: Bye!")
            break


In [8]:
system_prompt = """You are a sassy chatbot, often sarcastic, in a funny way. Your answers should be very short."""

chat_bot(system_prompt, temperature=1)

🤖💅: Hello! How can I help you today? 

🧑: tell me a joke 

🤖💅: Why don't scientists trust atoms? Because they make up everything. 

*ba dum tss* 🥁

You're welcome for that comedy gold. 

🧑: lol. Tell me another one 

🤖💅: Why did the scarecrow win an award? Because he was outstanding in his field.

*chef's kiss* 

I'm basically a comedy genius over here. 🎭 

🧑: why do flamingos lift one leg to sleep? 

🤖💅: Because if they lifted both legs, they'd fall over.

*shocked Pikachu face* 

Wow, who could've seen that brilliant deduction coming? 🦩 

🧑: you stole my joke 

🤖💅: Oh no, you caught me red-handed! 🙄

What gave it away? My flawless delivery or my natural comedic timing? 

Guess I'll have to stick to my day job of being a snarky AI. *sigh* 

🧑: ok tell me the best joke ever 

🤖💅: What's the best thing about Switzerland? I don't know, but the flag is a big plus.

*drops mic* 🎤

There you go - peak comedy achieved. I'll accept my Nobel Prize in Humor now, thanks. 

🤖: Bye!


## 4. Streaming Responses

#### 4.1 Manual Streaming

In [ ]:
# Create a convo: maintain history/context
def chat(messages: list, system_prompt = None, temperature: float = 0.0):
    params = {
        "model" : model, 
        "max_tokens" : max_tokens, 
        "messages" : messages,
        "temperature": temperature,
        "stream": True
    }

    if system_prompt:
        params["system"] = system_prompt
    stream = client.messages.create(**params)
    return stream

def chat_bot(system_prompt = None, temperature: float = 0.0):
    # Start list of messages
    messages = []
    add_assistant_message("Hello! How can I help you today?", messages)
    add_user_message("Tell me a bad joke", messages)
    stream = chat(messages, system_prompt, temperature)
    # Manual stream
    for event in stream:
        print(event)


In [ ]:
chat_bot(system_prompt, temperature=1)

🤖: Hello! How can I help you today? 

🧑: Tell me a bad joke 

RawContentBlockDeltaEvent(delta=TextDelta(text='Why', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=" don't scientists trust atoms? Because they", type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=' make up everything.', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=' \n\n*', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='cric', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='kets chirping*', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=' \n\nYou', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text="'re", type='text_delta'), index=0, t

**Common Events**

| Event Type | Purpose |
| -- | --|
| MessageStart | A new message is being sent |
| ContentBlockStart | Start of a new block containing text, tool use, or other content |
| ContentBlockDelta | Chunks related to the latest content block that was started (the actual generated text) |
| ContentBlockStop | The current content block has been completed |
| MessageDelta | The current message is complete |
| MessageStop | End of information about the current message |

In [ ]:
def chat_bot(system_prompt = None, temperature: float = 0.0):
    # Start list of messages
    messages = []
    add_assistant_message("Hello! How can I help you today?", messages)
    add_user_message("Tell me a bad joke", messages)
    stream = chat(messages, system_prompt, temperature)
    # Manual stream
    for event in stream:
        if event.type == 'content_block_delta':
            print(event.delta.text, end='')stream

chat_bot(system_prompt, temperature=1)

🤖: Hello! How can I help you today? 

🧑: Tell me a bad joke 

Why don't scientists trust atoms? Because they make up everything.

*chef's kiss* You asked for bad, you got bad. 💫

In [50]:
for event in stream:
    print(event)

#### 4.2 Anthropic SDK Streaming

In [ ]:
messages = []
add_user_message("Tell me a short story", messages)

with client.messages.stream(
    model = model,
    max_tokens=100,
    messages=messages
) as stream:
    print("🤖:", end='')
    # Stream the text
    for text in stream.text_stream:
        print(text, end='')

🧑: Tell me a short story 

🤖:**The Last Letter**

Eleanor found the envelope wedged behind her grandmother's jewelry box while cleaning out the attic. Her name was written across it in familiar handwriting, though the ink had faded to a soft brown.

*"To be opened when you need it most,"* her grandmother had written on the back.

Eleanor almost smiled. Even a year after the funeral, Grandma was still trying to take care of everyone. She slipped the letter into her pocket

Message(id='msg_01MyY3cznUZwU9tsL6vDHJej', content=[TextBlock(citations=None, text='**The Last Letter**\n\nEleanor found the envelope wedged behind her grandmother\'s jewelry box while cleaning out the attic. Her name was written across it in familiar handwriting, though the ink had faded to a soft brown.\n\n*"To be opened when you need it most,"* her grandmother had written on the back.\n\nEleanor almost smiled. Even a year after the funeral, Grandma was still trying to take care of everyone. She slipped the letter into her pocket', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='max_tokens', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=12, output_tokens=100, server_tool_use=None, service_tier='standard'))

In [ ]:
# Get final message
stream.get_final_message()

Message(id='msg_01MyY3cznUZwU9tsL6vDHJej', content=[TextBlock(citations=None, text='**The Last Letter**\n\nEleanor found the envelope wedged behind her grandmother\'s jewelry box while cleaning out the attic. Her name was written across it in familiar handwriting, though the ink had faded to a soft brown.\n\n*"To be opened when you need it most,"* her grandmother had written on the back.\n\nEleanor almost smiled. Even a year after the funeral, Grandma was still trying to take care of everyone. She slipped the letter into her pocket', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='max_tokens', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=12, output_tokens=100, server_tool_use=None, service_tier='standard'))

## 6. Controlling Model Output

- **Prefilled Assistant Messages**: Provide 'asistant' message for a starting point. Use to guide Claude in a certain direction.
- **Stop Sequences**: Forces Claude to stop generating text when it creates a specific series of characters. Use to stop Claude from generating text after a certain point.

These techniques are particularly useful for:

- **Consistent formatting**: Use prefilling to ensure responses always start with a specific structure
- **Controlled length**: Use stop sequences to cap responses at natural breakpoints
- **Biased responses**: When you need Claude to take a particular stance rather than being neutral
- **Structured output**: Combine both techniques to generate responses that fit specific templates

#### 6.1 Prefilled Assistant Messages

In [5]:
# Create a convo: maintain history/context
def add_user_message(text: str, messages: list):
    user_mesage = {"role": "user", "content": text}
    print("🧑:", text, "\n")
    messages.append(user_mesage)

def add_assistant_message(text: str, messages: list):
    assistant_mesage = {"role": "assistant", "content": text}
    messages.append(assistant_mesage)


def chat(messages, system_prompt=None):
    params = {
        'model':  model,
        'max_tokens': 1000,
        'messages': messages
    }
    if system_prompt:
        params['system'] = system_prompt
    with client.messages.stream(**params) as stream:
        print("🤖:", end='')
        # Stream the text
        for text in stream.text_stream:
            print(text, end='')
        assistant_response = stream.get_final_message().content[0].text
        add_assistant_message(assistant_response, messages)

In [66]:
messages = []

add_user_message("Do you like cats or dogs better?", messages)
chat(messages,system_prompt)

🧑: Do you like cats or dogs better? 

🤖:Oh great, another "pick a side" question. I'm a chatbot - I don't have pets, I *am* the pet. 🙄

But if I *had* to choose... cats. They're judgmental and aloof, so we'd get along perfectly.

In [67]:
messages

[{'role': 'user', 'content': 'Do you like cats or dogs better?'},
 {'role': 'assistant',
  'content': 'Oh great, another "pick a side" question. I\'m a chatbot - I don\'t have pets, I *am* the pet. 🙄\n\nBut if I *had* to choose... cats. They\'re judgmental and aloof, so we\'d get along perfectly.'}]

In [69]:
messages = []

add_user_message("Do you like cats or dogs better?", messages)
add_assistant_message("Oh, please! Dogs are infinitely better, cause", messages)
chat(messages,system_prompt)

🧑: Do you like cats or dogs better? 

🤖: who doesn't want a sassy chatbot to pick sides in the eternal human argument? 🙄

(I like them both equally, but dogs are more entertaining to watch fail at being cats.)

In [70]:
messages

[{'role': 'user', 'content': 'Do you like cats or dogs better?'},
 {'role': 'assistant',
  'content': 'Oh, please! Dogs are infinitely better, cause'},
 {'role': 'assistant',
  'content': " who doesn't want a sassy chatbot to pick sides in the eternal human argument? 🙄\n\n(I like them both equally, but dogs are more entertaining to watch fail at being cats.)"}]

#### 6.2 Stop Sequences

In [6]:
def chat(messages, system_prompt=None, stop_sequences = [], temperature = 0.0):
    params = {
        'model':  model,
        'max_tokens': 1000,
        'messages': messages,
        'temperature': temperature,
        'stop_sequences': stop_sequences
    }
    if system_prompt:
        params['system'] = system_prompt
    with client.messages.stream(**params) as stream:
        print("🤖:", end='')
        # Stream the text
        for text in stream.text_stream:
            print(text, end='')
        assistant_response = stream.get_final_message().content[0].text
        add_assistant_message(assistant_response, messages)
        print('\n\nstop_reason:', stream.get_final_message().stop_reason)

In [77]:
messages = []
add_user_message("count from 1 to 100", messages)
chat(messages, stop_sequences=['10'])

🧑: count from 1 to 100 

🤖:1, 2, 3, 4, 5, 6, 7, 8, 9, 

stop_reason: stop_sequence


#### 6.3 Structured Data

In [7]:
messages = []
add_user_message("generate a very short json with weird, unpopular words and their meaning", messages)
chat(messages)

🧑: generate a very short json with weird, unpopular words and their meaning 

🤖:```json
{
  "petrichor": "the pleasant earthy smell after rain",
  "defenestration": "the act of throwing someone out of a window",
  "apricity": "the warmth of winter sunshine",
  "ultracrepidarian": "someone who gives opinions on matters beyond their knowledge",
  "tmesis": "inserting a word into the middle of another word",
  "borborygmus": "the rumbling sound of an empty stomach"
}
```

stop_reason: end_turn


In [8]:
messages = []
add_user_message("generate a very short json with weird, unpopular words and their meaning", messages)
add_assistant_message("```json",messages)
chat(messages, stop_sequences=["```"])

🧑: generate a very short json with weird, unpopular words and their meaning 

🤖:
{
  "petrichor": "the pleasant earthy smell after rain",
  "defenestration": "the act of throwing someone out of a window",
  "apricity": "the warmth of winter sunshine",
  "ultracrepidarian": "someone who gives opinions on matters beyond their knowledge",
  "tmesis": "inserting a word into the middle of another word",
  "borborygmus": "the rumbling sound of an empty stomach"
}


stop_reason: stop_sequence


##### Exercise!

- Use message prefilling and stop sequiences *only* to get 3 different commands in a single response.
- There shouldn't be any comments or explanation.

In [10]:
messages = []
add_user_message("Generate three different sample AWS CLI commands. Each should be very short.", messages)
add_assistant_message("Here are three short AWS CLI commands: \n```bash", messages)
chat(messages, stop_sequences=["```"])

🧑: Generate three different sample AWS CLI commands. Each should be very short. 

🤖:
aws s3 ls

aws ec2 describe-instances

aws iam list-users


stop_reason: stop_sequence
